## 往期文章学习  
https://www.kesci.com/home/column/5cb43d67e0ad99002cad14d6  

### 本篇目录  
* 前言  
* 安装模块  
* 数据集介绍  
* 1、jieba 模块功能概要  
* 2、jieba 模块实战  
* 3、textrank4zh 模块实战  
* 4、snownlp 分词模块功能概要  
* 5、参考文献  
* -------------------  
### 从这个项目中，你能学到哪些技术知识？  
* jieba: 分词、提取关键词、词性标注  
* textrank4zh: 抽取关键词、关键短语和文本摘要  
* snownlp: 分词、汉字转拼音、繁体转简体、分句、摘要、关键词、相似度分析、情感分析、词性标注，tf和idf  


### 前言

分词是自然语言中最基础，也是必要环节来的。  
无论是提取关键词、相似度分析、词性标注或是以后运用算法进行训练预测等这些都是建立在文本分词的基础上。  
文本型和数值型数据是有区别的，如果不对文本进行分词切割，就不能很好地统计和识别文本的具体类别和具体含义  。  
文本型数据与数值型数据的区别和联系，等到下一节的'文本预处理'中再详细介绍  


### 安装模块

In [17]:
!pip install jieba

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
!pip install snownlp

    100% |████████████████████████████████| 37.6MB 15kB/s 
  Running setup.py bdist_wheel for snownlp ... - \ | / - \ | / - done
  Stored in directory: /home/kesci/.cache/pip/wheels/f3/81/25/7c197493bd7daf177016f1a951c5c3a53b1c7e9339fd11ec8f
Successfully built snownlp
You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
!pip install textrank4zh

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### 数据集介绍

In [20]:
# 查看当前挂载的数据集目录
!ls /home/kesci/work/xiaozhi

mydata.txt	     stopword.txt	  text_classification.zip
my_network_model.h5  text_classification


* 本文的数据集 mydata.txt 介绍：  
    * 1、这是小知同学本人在自己的头条号里面写的一篇文章，是关于爬虫学习路线的文章  
    * 2、把文章复制保存在 mydata.txt 数据集中，为了方便来理解中文分词是如何切分任意一篇中文文章的  
    * 3、数据集的原文链接：https://www.toutiao.com/i6611872532448412164/

### 1、jieba 分词模块功能概要 
* ----------------------- 
#### 第一块：分词模式和并行分词 
1. 精确分词 
试图将句子最精确地切开 
>jieba.cut('人生苦短，我学python',cut_all=False) 

2. 全模式 
把句子中所有的可能是词语的都扫描出来，速度非常快，但不能解决歧义 
>jieba.cut('人生苦短，我学python',cut_all=True) 

3. 搜索引擎模式 
在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词 
>jieba.cut_for_search('人生苦短，我学python') 

4. 用 lcut 生成 list 
>jieba.lcut 对 cut 的结果做了封装，l 代表 list，即返回的结果是一个 list 集合  
jieba.lcut('人生苦短，我学python',cut_all=False) 

5. 原理：将目标文本按行分隔后，把各行文本分配到多个 Python 进程并行分词， 
然后归并结果，从而获得分词速度的可观提升. 
基于 python 自带的 multiprocessing 模块，目前暂不支持 Windows 
用法： 
>jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数  
jieba.disable_parallel() # 关闭并行分词模式 
   
* ------------------ 
#### 第二块：关键词提取、词性标注、自定义字典 
6. 获取分词结果中词列表的 top n(关键词提取) 
>jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())    
jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=()) 

5. 获取词性 
>jieba.posseg 模块实现词性标注  
jieba.posseg.cut(sentence, cut_all=False) 

7. 自定义添加词和字典 
使用默认分词，是识别不出一句话中的新词，需要添加新词到字典 

自定义字典原理： 
* 词典格式和 dict.txt 一样，一个词占一行；每一行分三部分： 
* 词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒 
* file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码 

>D:/python/xiaozhi_life_dict.txt  
小知同学  
腾星火耀  
绿杨楼  
黄槐楼  
尚书院  

* jieba.load_userdict(file_name) # file_name 为文件类对象或自定义词典的路径 

完整示例 
>import jieba  
text = """  
小知同学时而在绿杨楼溜达，时而在黄槐楼遛狗，时而去尚书院阅读书籍，  
时而去腾星火耀办事处。  
"""  
jieba.load_userdict('D:/python/xiaozhi_life_dict.txt')  
data = jieba.cut(text)  # 默认精准模式  
print(','.join(data))  

4、还可以做： 
自动摘要、依存句法分析、情感分析等任务

### 2、jieba 模块实战  
#### 2.1 三种分词模式

In [21]:
# coding:utf-8
import jieba

sentence = open('/home/kesci/work/xiaozhi/mydata.txt', 'rb').read()

jieba.enable_parallel(4)  # 开启并行分词
# words = jieba.cut(sentence=sentence, cut_all=True)  # 全模式分词
words = jieba.cut(sentence=sentence, cut_all=False)  # 精确分词
# words = jieba.cut_for_search(sentence=sentence, HMM=True)  # 搜索引擎分词

print('/'.join(words))


大家/都/知道/，/学习/一门/学科/的/时候/是/要/清楚/它/的/知识/框架/才能/清晰/的/学习/、/有/系统/的/学习/，/下面/来列/一列/python/网络/爬虫/的/知识/框架/来/帮助/大家/能够/有效/的/学习/和/掌握/，/避免/不必要/的/坑/。/
/
/python/网络/爬虫/总的来说/有/五个/大/的/方面/：/
/前端/知识/—/—/基础/爬虫/—/—/框架/爬虫/—/—/分布式/爬虫/—/—/突破/反/爬虫/
/
/1/./前端/知识/：/
/“/网络/爬虫/”/很/明显/对象/是/网络/，/也/就是/网页/。/说/到/网页/，/这里/就/涉及/到/了/前端/的/知识/了/，/不过/大家/也/不要/慌/，/只要/懂点/必要/的/HTML5/框架/、/网页/的/http/请求/、/还有/JavaScript/、/css3/的/知识/就/可以/了/，/以/这样/的/水平/也/是/可以/学会/爬虫/的/啦/。/当然/，/如果/要/非常/精通/python/网络/爬虫/的话/，/深入/学习/前端/知识/是/必要/的/。/
/
/2/./基础/爬虫/：/
/（/1/）/基础/库/：/urllib/模块///requests/第三方/模块/
/
/首先/爬虫/就是/要/从/网页/上/把/我们/需要/的/信息/抓取/下来/的/，/那么/我们/就要/学习/urllib///requests/模块/，/这/两种/模块/是/负责/爬取/网页/的/。/这里/大家/觉得/哪/一种/用/的/习惯/就/用/哪/一种/，/选择/一种/精通/就/好/了/。/小编/推荐/读者/使用/使用/requests/模块/，/因为/这/一种/简便/很多/，/容易/操作/、/容易/理解/，/所以/requests/被/称为/“/人性化/模块/”/。/
/
/（/2/）/多/进程/、/多线程/和/协程/：/
/
/为什么/要学/着/三个/知识/呢/？/假如/你/要/爬/取/200/万条/的/数据/，/使用/一般/的/单/进程/或者/单线程/的话/，/你/爬/取/下载/这些/数据/，/也许/要/一个/星期/或是/更久/。/试问/这/是/你/想要/看到/的/结果/吗/？/显然/单/进程/和/单线程/不要/满足/我们/追求/的/高效率/，/太/浪费时间/了/。/只要/设置/好多/进程/和

In [22]:
# 分词后以列表形式返回
word_list = jieba.lcut(sentence=sentence)  # 默认精准分词
print(' '.join(word_list))

大家 都 知道 ， 学习 一门 学科 的 时候 是 要 清楚 它 的 知识 框架 才能 清晰 的 学习 、 有 系统 的 学习 ， 下面 来列 一列 python 网络 爬虫 的 知识 框架 来 帮助 大家 能够 有效 的 学习 和 掌握 ， 避免 不必要 的 坑 。 
 
 python 网络 爬虫 总的来说 有 五个 大 的 方面 ： 
 前端 知识 — — 基础 爬虫 — — 框架 爬虫 — — 分布式 爬虫 — — 突破 反 爬虫 
 
 1 . 前端 知识 ： 
 “ 网络 爬虫 ” 很 明显 对象 是 网络 ， 也 就是 网页 。 说 到 网页 ， 这里 就 涉及 到 了 前端 的 知识 了 ， 不过 大家 也 不要 慌 ， 只要 懂点 必要 的 HTML5 框架 、 网页 的 http 请求 、 还有 JavaScript 、 css3 的 知识 就 可以 了 ， 以 这样 的 水平 也 是 可以 学会 爬虫 的 啦 。 当然 ， 如果 要 非常 精通 python 网络 爬虫 的话 ， 深入 学习 前端 知识 是 必要 的 。 
 
 2 . 基础 爬虫 ： 
 （ 1 ） 基础 库 ： urllib 模块 / requests 第三方 模块 
 
 首先 爬虫 就是 要 从 网页 上 把 我们 需要 的 信息 抓取 下来 的 ， 那么 我们 就要 学习 urllib / requests 模块 ， 这 两种 模块 是 负责 爬取 网页 的 。 这里 大家 觉得 哪 一种 用 的 习惯 就 用 哪 一种 ， 选择 一种 精通 就 好 了 。 小编 推荐 读者 使用 使用 requests 模块 ， 因为 这 一种 简便 很多 ， 容易 操作 、 容易 理解 ， 所以 requests 被 称为 “ 人性化 模块 ” 。 
 
 （ 2 ） 多 进程 、 多线程 和 协程 ： 
 
 为什么 要学 着 三个 知识 呢 ？ 假如 你 要 爬 取 200 万条 的 数据 ， 使用 一般 的 单 进程 或者 单线程 的话 ， 你 爬 取 下载 这些 数据 ， 也许 要 一个 星期 或是 更久 。 试问 这 是 你 想要 看到 的 结果 吗 ？ 显然 单 进程 和 单线程 不要 满足 我们 追求 的 高效率 ， 太 浪费时间 了 。 只要 设置 好多 进程 和

#### 2.2 我们来做文本分析，提取文本中的关键词  

#### 通过提取出文档的关键词，我们可以了解到：  
* 文章属于什么类型文体，如：科技、生活、健康、环保(广义)  
* 文章大致说什么样的话题，如：科技类别中的某个知识点(狭义)  
* 可以控制不同词性的输出，如：只输出人名(可以知道这篇文章讲到了哪个人)

* 第一种文本分析方法  
    * jieba.analyse.textrank()

In [23]:
from jieba import analyse

sentence = open('/home/kesci/work/xiaozhi/mydata.txt', 'rb').read()
key_words = analyse.textrank(sentence=sentence, topK=20, withWeight=True, 
                             allowPOS=('n'))
key_words

[('爬虫', 1.0),
 ('框架', 0.48032277009554275),
 ('网页', 0.375903778558585),
 ('储存', 0.28989294150612954),
 ('模块', 0.28396439713396376),
 ('网络', 0.26928056210536605),
 ('分布式', 0.25213862134805176),
 ('数据', 0.24921712051780848),
 ('学会', 0.18787346572729172),
 ('进程', 0.17506500529136249),
 ('高效率', 0.17506500529136249),
 ('信息', 0.17506500529136249),
 ('协程', 0.17381538416008774),
 ('浪费时间', 0.17381538416008774),
 ('价值', 0.17381538416008774),
 ('时候', 0.1577095931982778),
 ('网站', 0.14990211449063331),
 ('措施', 0.13718889861223857),
 ('方式', 0.1311555495988141),
 ('朋友', 0.12911459351451918)]

* 第二种文本分析方法  
    * jieba.analyse.extract_tags()

In [24]:
from jieba import analyse

sentence = open('/home/kesci/work/xiaozhi/mydata.txt', 'rb').read()
''' 
1、采用默认的情况下 
topK: 默认输出排名靠前的 20 个高频词(关键词)
withWeight: 是否一起输出词的权重，默认不输出权重
allowPOS: 筛选什么样的词性输出，默认输出所有
'''
key_words = analyse.extract_tags(sentence=sentence, topK=20, withWeight=False, 
                                 allowPOS=())
key_words

['爬虫',
 '网页',
 'python',
 '框架',
 '知识',
 '模块',
 '储存',
 'requests',
 '网络',
 '分布式',
 '学习',
 '爬取',
 'MongoDB',
 '前端',
 '三种',
 '数据',
 '多线程',
 '代理服务器',
 '进程',
 '设置']

In [25]:
''' 2、只输出 10 个关键词 '''
key_words = analyse.extract_tags(sentence=sentence, topK=10)
key_words

['爬虫', '网页', 'python', '框架', '知识', '模块', '储存', 'requests', '网络', '分布式']

In [26]:
''' 3、输出关键词的权重 '''
key_words = analyse.extract_tags(sentence=sentence, topK=10, withWeight=True)
key_words

[('爬虫', 0.576025142312844),
 ('网页', 0.19674556925594036),
 ('python', 0.19193434064288992),
 ('框架', 0.16611620656215598),
 ('知识', 0.1331185750895642),
 ('模块', 0.12328947563353213),
 ('储存', 0.11111941575137615),
 ('requests', 0.10967676608165138),
 ('网络', 0.09969344697511467),
 ('分布式', 0.09517722135229358)]

In [27]:
''' 4、设置词性，筛选出想要的关键词 '''
key_words = analyse.extract_tags(sentence=sentence, topK=10, withWeight=True, 
                                 allowPOS=('v'))  # 在这里我指定只提取动词的关键词
# 有兴趣可以去试一试其他的词性
key_words


[('爬取', 0.703221617817647),
 ('知识', 0.5690166543044118),
 ('学习', 0.39646900704284316),
 ('使用', 0.22138725209754903),
 ('抓取', 0.22081608475294118),
 ('操作', 0.21603219419098038),
 ('需要', 0.2129647374264706),
 ('下载', 0.21024727281852942),
 ('请求', 0.2092673531461765),
 ('掌握', 0.18116407535117648)]

#### 其他词性对应的字母如下：  
* 该内容来源于网络  
* --------------------------------  
#### ICTCLAS 汉语词性标注集  
#### 代码	名称	帮助记忆的诠释  
* Ag	形语素	形容词性语素。形容词代码为a，语素代码ｇ前面置以A。  
* a	形容词	取英语形容词adjective的第1个字母。  
* ad	副形词	直接作状语的形容词。形容词代码a和副词代码d并在一起。  
* an	名形词	具有名词功能的形容词。形容词代码a和名词代码n并在一起。  
* b	区别词	取汉字“别”的声母。  
* c	连词	取英语连词conjunction的第1个字母。  
* Dg	副语素	副词性语素。副词代码为d，语素代码ｇ前面置以D。  
* d	副词	取adverb的第2个字母，因其第1个字母已用于形容词。  
* e	叹词	取英语叹词exclamation的第1个字母。  
* f	方位词	取汉字“方” 的声母。  
* g	语素	绝大多数语素都能作为合成词的“词根”，取汉字“根”的声母。  
* h	前接成分	取英语head的第1个字母。  
* i	成语	取英语成语idiom的第1个字母。  
* j	简称略语	取汉字“简”的声母。  
* k	后接成分	 
* l	习用语	习用语尚未成为成语，有点“临时性”，取“临”的声母。  
* m	数词	取英语numeral的第3个字母，n，u已有他用。  
* Ng	名语素	名词性语素。名词代码为n，语素代码ｇ前面置以N。  
* n	名词	取英语名词noun的第1个字母。  
* nr	人名	名词代码n和“人(ren)”的声母并在一起。  
* ns	地名	名词代码n和处所词代码s并在一起。  
* nt	机构团体	“团”的声母为t，名词代码n和t并在一起。  
* nz	其他专名	“专”的声母的第1个字母为z，名词代码n和z并在一起。  
* o	拟声词	取英语拟声词onomatopoeia的第1个字母。  
* p	介词	取英语介词prepositional的第1个字母。  
* q	量词	取英语quantity的第1个字母。  
* r	代词	取英语代词pronoun的第2个字母,因p已用于介词。  
* s	处所词	取英语space的第1个字母。  
* Tg	时语素	时间词性语素。时间词代码为t,在语素的代码g前面置以T。  
* t	时间词	取英语time的第1个字母。  
* u	助词	取英语助词auxiliary 的第2个字母,因a已用于形容词。  
* Vg	动语素	动词性语素。动词代码为v。在语素的代码g前面置以V。  
* v	动词	取英语动词verb的第一个字母。  
* vd	副动词	直接作状语的动词。动词和副词的代码并在一起。  
* vn	名动词	指具有名词功能的动词。动词和名词的代码并在一起。  
* w	标点符号	 
* x	非语素字	非语素字只是一个符号，字母x通常用于代表未知数、符号。  
* y	语气词	取汉字“语”的声母。  
* z	状态词	取汉字“状”的声母的前一个字母。

#### 2.3 词性标注

In [28]:
from jieba import posseg

sentence = open('/home/kesci/work/xiaozhi/mydata.txt', 'rb').read()
# print('文本解码>>>\n', sentence.decode('gbk')[:89])

words = posseg.cut(sentence)
word_flag, i = [], 0
for word, flag in words:
    i += 1
    word_flag.append((word, flag))
    if i <= 10:
        print(word, flag)
print(word_flag[:10])

大家 n
都 d
知道 v
， x
学习 v
一门 m
学科 n
的 uj
时候 n
是 v
[('大家', 'n'), ('都', 'd'), ('知道', 'v'), ('，', 'x'), ('学习', 'v'), ('一门', 'm'), ('学科', 'n'), ('的', 'uj'), ('时候', 'n'), ('是', 'v')]


### 3、textrank4zh 模块功能介绍和使用  
* 自动从文本中抽取关键词、关键短语和文本摘要

#### 3-1、抽取关键词、关键短语 
* 函数以及参数介绍 
* tr4w.get_keywords(20, word_min_len=1)                    # 提取关键字 
    >Signature: tr4w.get_keywords(num=6, word_min_len=1)  
    Docstring:  
    获取最重要的num个长度大于等于word_min_len的关键词。  
    Return:  
    关键词列表。  
* tr4w.get_keyphrases(keywords_num=20, min_occur_num= 2)   # 提取关键短语 
    >Signature: tr4w.get_keyphrases(keywords_num=12, min_occur_num=2)  
    Docstring:  
    获取关键短语。  
    获取 keywords_num 个关键词构造的可能出现的短语，要求这个短语在原文本中至少出现的次数为min_occur_num。  
    Return:  
    关键短语的列表。

In [29]:
from textrank4zh import TextRank4Keyword 

''' 官方说法：py2 中 text 必须是 utf8 编码的 str 或者 unicode 对象，
              py3 中必须是 utf8 编码的 bytes 或者 str 对象
    实际运用：在本案例中，使用 'utf-8' 不能正常运行，而采用 'gbk' 编码可以正常运行，
              所以大家看着情况灵活运用吧 
'''
text = open('/home/kesci/work/xiaozhi/mydata.txt', 'r', encoding='gbk').read()
tr4w = TextRank4Keyword()
tr4w.analyze(text=text, lower=True, window=2)  # lower 是否将文本转换为小写

# ------------------------------------------------------
print('提取关键词>>>\n', '---' * 15)
for item in tr4w.get_keywords(num=10, word_min_len=1):
    print(item)
    print(item.word, item['weight'])  # 关键词、对应的权重(字典操作)

# ------------------------------------------------------
print('\n提取关键短语>>>\n', '---' * 15)
for item in tr4w.get_keyphrases(keywords_num=20, min_occur_num=2):
    print(item)                    # 关键短语


提取关键词>>>
 ---------------------------------------------
{'word': '爬虫', 'weight': 0.03936331625458298}
爬虫 0.03936331625458298
{'word': '框架', 'weight': 0.025920263885343478}
框架 0.025920263885343478
{'word': '网页', 'weight': 0.025429356795653592}
网页 0.025429356795653592
{'word': '使用', 'weight': 0.018341125509380565}
使用 0.018341125509380565
{'word': '学习', 'weight': 0.016347304362054803}
学习 0.016347304362054803
{'word': '需要', 'weight': 0.01545757433752497}
需要 0.01545757433752497
{'word': '信息', 'weight': 0.014839375107394526}
信息 0.014839375107394526
{'word': '储存', 'weight': 0.012989611470711638}
储存 0.012989611470711638
{'word': '知识', 'weight': 0.01245963506364818}
知识 0.01245963506364818
{'word': 'python', 'weight': 0.012371394251488572}
python 0.012371394251488572

提取关键短语>>>
 ---------------------------------------------
框架爬虫
知识框架
爬取网页


#### 3-2、抽取文本摘要  
* 1、目前主流的文本摘要方式  
* 一种是抽取式（extractive）  
    * 概念：  
        * 抽取式顾名思义，就是按照一定的权重，从原文中寻找跟中心思想最接近的一条或几条句子。  
    * 效果：  
        * 抽取式的摘要目前已经比较成熟，但是抽取质量及内容流畅度不是那么理想  

* 另一种是生成式（abstractive）  
    * 概念：  
        * 生成式则是计算机通读原文后，在理解整篇文章意思的基础上，按自己的话生成流畅的翻译。  
    * 效果：  
        * 伴随着深度学习的研究，生成式摘要的质量和流畅度都有很大的提升，  
        * 但目前也受到原文本长度过长、抽取内容不佳等的限制。  
        
* -------------------  
本案例使用抽取式方法  
* 2、函数以及参数介绍  
* tr4s.get_key_sentences(num=3)  
    >Signature: tr4s.get_key_sentences(num=6, sentence_min_len=6)  
    Docstring:  
    获取最重要的num个长度大于等于sentence_min_len的句子用来生成摘要。  
    

In [30]:
from textrank4zh import TextRank4Sentence

text = open('/home/kesci/work/xiaozhi/mydata.txt', 'r', encoding='gbk').read()
tr4s = TextRank4Sentence()
tr4s.analyze(text=text, lower=True, source='all_filters') 
'''
source ——> 生成句子之间的相似度,默认值为`'all_filters'`，
            可选值为`'no_filter', 'no_stop_words'
'''
 
# ------------------------------------------------
print('提取文本摘要>>>\n', '---' * 20)
for item in tr4s.get_key_sentences(num=3):
    print(item)
    print(item.index, '\t', item.weight, '\t', item['sentence'], '\n')

提取文本摘要>>>
 ------------------------------------------------------------
{'index': 23, 'sentence': '在爬取网页的时候有时会失败，因为别人网站设置了反爬虫措施了，这个时候就需要我们去伪装自己的行为，让对方网站察觉不到我们就是爬虫方', 'weight': 0.04319642085085014}
23 	 0.04319642085085014 	 在爬取网页的时候有时会失败，因为别人网站设置了反爬虫措施了，这个时候就需要我们去伪装自己的行为，让对方网站察觉不到我们就是爬虫方 

{'index': 9, 'sentence': '首先爬虫就是要从网页上把我们需要的信息抓取下来的，那么我们就要学习urllib/requests模块，这两种模块是负责爬取网页的', 'weight': 0.04222076414544854}
9 	 0.04222076414544854 	 首先爬虫就是要从网页上把我们需要的信息抓取下来的，那么我们就要学习urllib/requests模块，这两种模块是负责爬取网页的 

{'index': 0, 'sentence': '大家都知道，学习一门学科的时候是要清楚它的知识框架才能清晰的学习、有系统的学习，下面来列一列python网络爬虫的知识框架来帮助大家能够有效的学习和掌握，避免不必要的坑', 'weight': 0.04108120739367898}
0 	 0.04108120739367898 	 大家都知道，学习一门学科的时候是要清楚它的知识框架才能清晰的学习、有系统的学习，下面来列一列python网络爬虫的知识框架来帮助大家能够有效的学习和掌握，避免不必要的坑 



### 4、snownlp 模块  
#### 4.1 snownlp 模块功能概要

In [31]:
# ------------------------------------------------------
# 第一块：总览其功能
# from snownlp import SnowNLP  
# s = SnowNLP()
# SnowNLP类：  
# s.words               # 功能1 分词  
# s.pinyin              # 功能2 汉字转拼音 
# s.han                 # 功能3 繁体转简体
# s.tags                # 功能4 词性 
# s.sentences           # 功能5 分句  
# s.sentiments          # 功能6 情感度  
# s.keywords(limit=10)  # 功能7 关键词  
# s.summary(10          # 功能8 文本摘要
# s.sim()               # 功能9 相似度分析  
# s.tf                  # 功能10 tf = 某个词在本文档中出现的次数/本文档的总词数   
# s.idf                 # 功能10 idf = log(语料库文档总数/(包含该词的文档数+1))  


#### 4.2 snownlp 实战训练

In [32]:
from snownlp import SnowNLP

# 读取任意一篇中文文档
text = open('/home/kesci/work/xiaozhi/mydata.txt', encoding='gbk').read()
s = SnowNLP(text)


In [33]:
# 1、分词(效果不够 jieba 的分词效果好)
s.words[:20]

['大家',
 '都',
 '知道',
 '，',
 '学习',
 '一',
 '门',
 '学科',
 '的',
 '时候',
 '是',
 '要',
 '清楚',
 '它',
 '的',
 '知识',
 '框架',
 '才',
 '能',
 '清晰']

In [34]:
# 2、汉字转拼音 
s.pinyin[:20]

['da',
 'jia',
 'dou',
 'zhi',
 'dao',
 '，',
 'xue',
 'xi',
 'yi',
 'men',
 'xue',
 'ke',
 'de',
 'shi',
 'hou',
 'shi',
 'yao',
 'qing',
 'chu',
 'ta']

In [35]:
# 3、繁体转简体
fj = SnowNLP(u'「繁體字」「繁體中文」的叫法在臺灣亦很常見。')  
fj.han 

'「繁体字」「繁体中文」的叫法在台湾亦很常见。'

In [36]:
# 4、词性标注(效果不够 jieba 的效果好)  
k = 0
for i in s.tags:
    k += 1
    if k <= 20:
        print(i)

('大家', 'r')
('都', 'd')
('知道', 'v')
('，', 'w')
('学习', 'v')
('一', 'm')
('门', 'q')
('学科', 'n')
('的', 'u')
('时候', 'n')
('是', 'v')
('要', 'v')
('清楚', 'a')
('它', 'r')
('的', 'u')
('知识', 'n')
('框架', 'n')
('才', 'd')
('能', 'v')
('清晰', 'a')


In [37]:
# 5、分句(在停用词逗号、句号等分隔符处分句)  
sentence = s.sentences[:20]     
sentence

['大家都知道',
 '学习一门学科的时候是要清楚它的知识框架才能清晰的学习、有系统的学习',
 '下面来列一列python网络爬虫的知识框架来帮助大家能够有效的学习和掌握',
 '避免不必要的坑',
 'python网络爬虫总的来说有五个大的方面：',
 '前端知识——基础爬虫——框架爬虫——分布式爬虫——突破反爬虫',
 '1.前端知识：',
 '“网络爬虫”很明显对象是网络',
 '也就是网页',
 '说到网页',
 '这里就涉及到了前端的知识了',
 '不过大家也不要慌',
 '只要懂点必要的HTML5框架、网页的http请求、还有JavaScript、css3的知识就可以了',
 '以这样的水平也是可以学会爬虫的啦',
 '当然',
 '如果要非常精通python网络爬虫的话',
 '深入学习前端知识是必要的',
 '2.基础爬虫：',
 '（1）基础库：urllib模块/requests第三方模块',
 '首先爬虫就是要从网页上把我们需要的信息抓取下来的']

In [38]:
# 6、情感分析(注意目前 snownlp 模块只针对买卖东西时的评价，其他方面的效果不好)  
'''s = SnowNLP('这个商品很好用')  
s.sentiments             # 0.7492624097842814 积极的概率(一般大于0.5)  
s = SnowNLP('这个商品很垃圾')  
s.sentiments             # 0.18571145823186264 消极的概率(一般小于0.5)
'''

# 说明：这篇文章不是买卖东西时的评论语来的，所以预测效果会不好的
# 这里是为了告诉读者，它这个方法是怎样使用的
for item in sentence:
    print(SnowNLP(item).sentiments)

0.5512110016420362
0.9996878003759021
0.9756245505565548
0.05348175336301941
0.4263989249763027
0.9999719421657834
0.9030141393768442
0.7967692495130689
0.3999999999999998
0.27766712995735
0.9798495749719074
0.34527875471283087
0.9444096163288037
0.7388876001043244
0.5262327818078083
0.5688291481104605
0.9866935756029875
0.6294192655653609
0.10200483072227873
0.23933045688083876


In [39]:
# 7、提取关键词(效果不够 jieba 的效果好) 
s.keywords(20)

['爬虫',
 '网页',
 '不',
 '框架',
 '种',
 '知识',
 '储存',
 '数据',
 '模',
 '三',
 '式',
 '很',
 '库',
 'python',
 '信息',
 '代理',
 '块',
 '设置',
 '学习',
 '网络']

In [40]:
# 8、抽取文本摘要
s.summary(20)

['以上就是python网络爬虫的从入门到精通的知识框架',
 '下面来列一列python网络爬虫的知识框架来帮助大家能够有效的学习和掌握',
 '这两种模块是负责爬取网页的',
 '框架不止这两种',
 '首先爬虫就是要从网页上把我们需要的信息抓取下来的',
 '数据的储存大概就这三种方式了',
 '前端知识——基础爬虫——框架爬虫——分布式爬虫——突破反爬虫',
 '学会这三种灵活运用会很方便的',
 '这里有很多并不是我们想要的信息',
 '如果要非常精通python网络爬虫的话',
 '三种在不同的场景各有特色也各有不足',
 '分布式爬虫就是运用了MongoDB来储存的',
 '让对方网站察觉不到我们就是爬虫方',
 '深入学习前端知识是必要的',
 '“网络爬虫”很明显对象是网络',
 'MongoDB在爬虫里是非常好的储存方式',
 '只要懂点必要的HTML5框架、网页的http请求、还有JavaScript、css3的知识就可以了',
 '显然单进程和单线程不要满足我们追求的高效率',
 'python网络爬虫总的来说有五个大的方面：',
 '这里有三种解析器']

In [41]:
# 9、相似度分析  
s_sim = SnowNLP([
                 [u'这篇', u'文章', '写作'],  
                 [u'那篇', u'论文', '学术'],  
                 [u'这个', '小知同学', '小知同学']
                ])
           
s_sim.sim([u'小知同学'])        


[0, 0, 0.7297508910942724]

In [42]:
# 10、tf + idf
s_sim.tf

[{'这篇': 1, '文章': 1, '写作': 1},
 {'那篇': 1, '论文': 1, '学术': 1},
 {'这个': 1, '小知同学': 2}]

In [43]:
s_sim.idf

{'这篇': 0.5108256237659907,
 '文章': 0.5108256237659907,
 '写作': 0.5108256237659907,
 '那篇': 0.5108256237659907,
 '论文': 0.5108256237659907,
 '学术': 0.5108256237659907,
 '这个': 0.5108256237659907,
 '小知同学': 0.5108256237659907}

### 5、参考文献

jieba模块相关：  
>官网文档：https://github.com/fxsjy/jieba  

textrank4zh 模块相关：  
>官网文档：https://github.com/letiantian/TextRank4ZH  

snownlp 模块相关：  
>官网文档：https://github.com/isnowfy/snownlp  

文本摘要相关：  
>https://blog.csdn.net/qq_41853758/article/details/82859654